In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("train.csv")
df = pd.read_csv("train.csv")
df.tail()

,ID,TargetID,DRUGID,DRUGTYPE,Drug_high_status,DRUGNAME,PUBCHCID,Disease_of_highest_status,Drug_Status,UNIPROID,TARGNAME,GENENAME,SYNONYMS,FUNCTION,BIOCLASS,SEQUENCE,Disease,Accession Number,Target_Status
134481,147910,T59328,D06WRF,Small molecular drug,Phase 2,KD019,10458325,Meningioma metastases,Phase 2,EGFR_HUMAN,Epidermal growth factor receptor (EGFR),EGFR,Receptor tyrosine-protein kinase erbB-1; Proto...,Receptor tyrosine kinase binding ligands of th...,Kinase,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,Chronic pain,P00533,Phase 2
134482,37688,T65236,D0WM5O,Small molecular drug,Phase 1/2,G1,5322399,Lymphoma,Phase 1/2,GPER1_HUMAN,G-protein coupled estrogen receptor 1 (GPER1),GPER1,Membrane estrogen receptor; Lymphocyte-derived...,Receptor for estrogen. .,GPCR rhodopsin,MDVTSQARGVGLEMYPGTAQPAAPNTTSPELNLSHPLLGTALANGT...,Lymphoma,Q99527,Phase 1/2
134483,30073,T47768,D04JHN,Small molecular drug,Approved,Hydromorphone,5284570,Chronic pain,Phase 3,OPRM_HUMAN,Opioid receptor mu (MOP),OPRM1,hMOP; Mu-type opioid receptor; Mu opioid recep...,Receptor for natural and synthetic opioids inc...,GPCR rhodopsin,MDSSAAPTNASNCTDALAYSSCSPAPSPGSWVNLSHLDGNLSDPCG...,Movement disorder,P35372,Investigative
134484,37855,T40276,D0O0LS,Small molecular drug,Approved,Entrectinib,25141092,Mammary analogue secretory carcinoma,Phase 1,KPCB_HUMAN,Protein kinase C beta (PRKCB),PRKCB,Protein kinase C beta type; PRKCB1; PKCB; PKC-...,Plays a key role in B-cell activation by regul...,Kinase,MADPAAGPPPSEGEESTVRFARKGALRQKNVHEVKNHKFTARFFKQ...,Malignant haematopoietic neoplasm,P05771,Phase 3
134485,3288,T68251,D00UCC,Small molecular drug,Discontinued in Phase 1,ABT-518,9827497,Solid tumour/cancer,Discontinued in Phase 1,MMP2_HUMAN,Matrix metalloproteinase-2 (MMP-2),MMP2,TBE-1; Matrix metalloproteinase 2; CLG4A; 72 k...,As well as degrading extracellular matrix prot...,Peptidase,MEALMARGALTGPLRALCLLGCLLSHAAAAPSPIIKFPGDVAPKTD...,Multiple sclerosis,P08253,Terminated


In [3]:
df["Target_Status"].value_counts()

Target_Status
Approved                           31031
Phase 2                            29621
Phase 1                            16912
Phase 3                            15514
Investigative                       9174
Phase 1/2                           6160
Discontinued in Phase 2             5815
Terminated                          5234
Patented                            3055
Discontinued in Phase 3             2510
Discontinued in Phase 1             2379
Preclinical                         1978
Withdrawn from market               1519
Phase 2/3                           1432
Phase 4                              507
Clinical trial                       348
Preregistration                      228
Phase 1b                             157
Phase 2a                             135
Discontinued in Preregistration      131
Discontinued in Phase 1/2            129
Registered                           121
Approved (orphan drug)               105
Application submitted                 80
IN

In [4]:
print(df.columns)
to_remove = ["ID","SEQUENCE","FUNCTION","SYNONYMS","UNIPROID","PUBCHCID","Accession Number"]
df.drop(columns=to_remove,inplace=True)
print(df.columns)

Index(['ID', 'TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 'DRUGNAME',
       'PUBCHCID', 'Disease_of_highest_status', 'Drug_Status', 'UNIPROID',
       'TARGNAME', 'GENENAME', 'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE',
       'Disease', 'Accession Number', 'Target_Status'],
      dtype='object')
Index(['TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 'DRUGNAME',
       'Disease_of_highest_status', 'Drug_Status', 'TARGNAME', 'GENENAME',
       'BIOCLASS', 'Disease', 'Target_Status'],
      dtype='object')


In [5]:
from sklearn.preprocessing import LabelEncoder
Target_Status_encoded = LabelEncoder()
df["Target_Status_encoded"] = Target_Status_encoded.fit_transform(df["Target_Status"])
print(df[["Target_Status", "Target_Status_encoded"]])
df_encoded = pd.get_dummies(df.drop(columns=['Target_Status']), drop_first=True)
df_encoded['Target_Status'] = df['Target_Status']
print(df_encoded)
df_encoded.drop(columns=['TargetID_T00156'],inplace = True)


        Target_Status  Target_Status_encoded
0          Terminated                     31
1            Approved                      1
2            Approved                      1
3             Phase 2                     22
4            Approved                      1
...               ...                    ...
134481        Phase 2                     22
134482      Phase 1/2                     18
134483  Investigative                     14
134484        Phase 3                     26
134485     Terminated                     31

[134486 rows x 2 columns]
        Target_Status_encoded  TargetID_T00156  TargetID_T00176  \
0                          31            False            False   
1                           1            False            False   
2                           1            False            False   
3                          22            False            False   
4                           1            False            False   
...                       ...  

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.metrics import accuracy_score


X = df_encoded.drop(columns=['Target_Status'])  # Features
y = df_encoded['Target_Status']                   # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = RandomForestClassifier(random_state=42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


Accuracy: 1.00


In [7]:
y_pred = model.predict(X_test)
import pandas as pd
print(y.value_counts())

test_df = pd.read_csv('test.csv')

to_remove = ["SEQUENCE", "FUNCTION", "SYNONYMS", "UNIPROID", "PUBCHCID", "Accession Number"]
test_df_cleaned = test_df.drop(columns=to_remove)

test_df_encoded = pd.get_dummies(test_df_cleaned, drop_first=True)

X_test_final = test_df_encoded.reindex(columns=X.columns, fill_value=0)

predictions = model.predict(X_test_final)

submission = pd.DataFrame({
    'ID': test_df['ID'], 
    'Target_Status': predictions
})
print(submission["Target_Status"].value_counts())
submission.to_csv('submission1.csv', index=False)

print("Submission file created: submission.csv")



Target_Status
Approved                           31031
Phase 2                            29621
Phase 1                            16912
Phase 3                            15514
Investigative                       9174
Phase 1/2                           6160
Discontinued in Phase 2             5815
Terminated                          5234
Patented                            3055
Discontinued in Phase 3             2510
Discontinued in Phase 1             2379
Preclinical                         1978
Withdrawn from market               1519
Phase 2/3                           1432
Phase 4                              507
Clinical trial                       348
Preregistration                      228
Phase 1b                             157
Phase 2a                             135
Discontinued in Preregistration      131
Discontinued in Phase 1/2            129
Registered                           121
Approved (orphan drug)               105
Application submitted                 80
IN